In [1]:
# -*- coding: utf-8 -*-
"""
File này thực hiện:
1. Tải dữ liệu từ pipeline feature engineering nâng cao.
2. Sử dụng Optuna để tìm siêu tham số tốt nhất cho LightGBM.
3. Huấn luyện lại mô hình tốt nhất trên toàn bộ dữ liệu train.
4. Tạo file submission cuối cùng.
"""

import pandas as pd
import numpy as np
import os
import warnings
import lightgbm as lgb
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# =============================================================================
# 1. TẢI DỮ LIỆU
# =============================================================================
print("--- BƯỚC 1: TẢI DỮ LIỆU ĐÃ XỬ LÝ NÂNG CAO ---")
try:
    save_dir = "Data_clean_v3"
    train_df = pd.read_csv(f'{save_dir}/train_cleaned.csv')
    test_df = pd.read_csv(f'{save_dir}/test_cleaned.csv')
    
    X = train_df.drop(columns=['Survived', 'PassengerId'])
    y = train_df['Survived']
    X_test = test_df.drop(columns=['PassengerId'])
    
    # Đảm bảo các cột của X_test khớp với X
    X_test = X_test[X.columns]
    
    print(f"Tải dữ liệu thành công. Số đặc trưng: {X.shape[1]}")
except Exception as e:
    print(f"Lỗi khi tải dữ liệu: {e}. Hãy chạy file pipeline_advanced_fe.py trước.")
    exit()

# =============================================================================
# 2. TỐI ƯU HÓA SIÊU THAM SỐ VỚI OPTUNA
# =============================================================================
print("\n--- BƯỚC 2: TỐI ƯU HÓA SIÊU THAM SỐ CHO LIGHTGBM VỚI OPTUNA ---")

def objective(trial):
    """Hàm mục tiêu mà Optuna sẽ cố gắng tối ưu hóa."""
    
    # Định nghĩa không gian tìm kiếm siêu tham số
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    # Đánh giá mô hình bằng cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = lgb.LGBMClassifier(**param)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                  callbacks=[lgb.early_stopping(100, verbose=False)])
        
        preds = model.predict(X_val)
        scores.append(f1_score(y_val, preds, average='weighted'))

    return np.mean(scores)

# Tạo một study và bắt đầu tối ưu hóa
# `direction='maximize'` vì chúng ta muốn f1-score cao nhất
study = optuna.create_study(direction='maximize', study_name='LGBM Optimization')
study.optimize(objective, n_trials=50) # Chạy 50 lần thử nghiệm

best_params = study.best_params
print("\nQuá trình tối ưu hóa hoàn tất!")
print(f"Điểm F1-score tốt nhất (cross-validated): {study.best_value:.5f}")
print("Các siêu tham số tốt nhất được tìm thấy:")
print(best_params)

# =============================================================================
# 3. HUẤN LUYỆN LẠI VÀ TẠO SUBMISSION
# =============================================================================
print("\n--- BƯỚC 3: HUẤN LUYỆN LẠI VÀ TẠO FILE SUBMISSION ---")

# Tạo mô hình cuối cùng với các tham số tốt nhất
final_model = lgb.LGBMClassifier(objective='binary', verbosity=-1, **best_params)

# Huấn luyện trên toàn bộ dữ liệu train
final_model.fit(X, y)
print("Đã huấn luyện xong mô hình cuối cùng trên toàn bộ dữ liệu.")

# Dự đoán trên tập test
predictions = final_model.predict(X_test)

# Tạo và lưu file submission
submission_df = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': predictions})
# --- Lưu trữ theo cấu trúc của bạn ---
save_dir = "Pred_Result"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    print(f"Đã tạo thư mục '{save_dir}'")

try:

    # 2. Lưu các file CSV đã được làm sạch hoàn chỉnh
    submission_df.to_csv(f'{save_dir}/pred_result_Lab_1_v6.csv', index=False)
    print(f"- Đã lưu pred_result_Lab_1_v6 vào '{save_dir}'")

    print("\nNội dung thư mục 'Pred_Result':")
    print(os.listdir(save_dir))

except Exception as e:
    print(f"\nLỗi khi lưu file: {e}")

print("\nHoàn tất!")
print("File submission 'advanced_submission.csv' đã được tạo.")
print("5 dòng đầu của file submission:")
print(submission_df.head())

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- BƯỚC 1: TẢI DỮ LIỆU ĐÃ XỬ LÝ NÂNG CAO ---
Tải dữ liệu thành công. Số đặc trưng: 29

--- BƯỚC 2: TỐI ƯU HÓA SIÊU THAM SỐ CHO LIGHTGBM VỚI OPTUNA ---

Quá trình tối ưu hóa hoàn tất!
Điểm F1-score tốt nhất (cross-validated): 0.84356
Các siêu tham số tốt nhất được tìm thấy:
{'n_estimators': 400, 'learning_rate': 0.04297164476413649, 'num_leaves': 219, 'max_depth': 7, 'lambda_l1': 0.409415244368732, 'lambda_l2': 4.839809785911123e-05, 'feature_fraction': 0.9058846679882524, 'bagging_fraction': 0.7832336385408966, 'bagging_freq': 2}

--- BƯỚC 3: HUẤN LUYỆN LẠI VÀ TẠO FILE SUBMISSION ---
Đã huấn luyện xong mô hình cuối cùng trên toàn bộ dữ liệu.
- Đã lưu pred_result_Lab_1_v6 vào 'Pred_Result'

Nội dung thư mục 'Pred_Result':
['pred_result_Lab_1.csv', 'pred_result_Lab_1_v2.csv', 'pred_result_Lab_1_v3.csv', 'pred_result_Lab_1_v4.csv', 'pred_result_Lab_1_v5.csv', 'pred_result_Lab_1_v6.csv']

Hoàn tất!
File submission 'advanced_submission.csv' đã được tạo.
5 dòng đầu của file submission:
   P